#### Notes

1.   If we are providing a solution for link prediction, we might consider using imbalanced data with more negative samples to mimic the real world scenario. 
2.   We can think of a better approach than randomly selecting edges and checking if they are +ve/-ve. We can do this by factoring in a weight related to the path between two nodes.

In [1]:
!pip install StellarGraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 4.9 MB/s 


#Imports + Installations

In [2]:
# Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd # Pandas to create small dataframes 
import datetime # Convert to unix time
import time # Convert to unix time
# If numpy is not installed already : pip3 install numpy
import numpy as np # Do aritmetic operations on arrays
# Matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns # Plots
from matplotlib import rcParams # Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans # Clustering
import math
import pickle
import os
# To install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from tqdm.notebook import tqdm
import os
import random
from sklearn.model_selection import train_test_split

import collections
from stellargraph.mapper import GraphWaveGenerator
from stellargraph import StellarGraph
from scipy.sparse.linalg import eigs

#Loading the data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
os.listdir()

['.config', 'gdrive', 'sample_data']

In [5]:
os.listdir('gdrive/My Drive/Twitch Link Prediction/ES')

['musae_ES_edges.csv', 'musae_ES_target.csv', 'musae_ES_features.json']

In [6]:
data_path = 'gdrive/My Drive/Twitch Link Prediction/ES'

In [7]:
train = pd.read_csv(data_path+'/musae_ES_edges.csv', names = ['Source', 'Destination'])[1:]
train['Source'] = train['Source'].astype('int64')
train['Destination'] = train['Destination'].astype('int64')
train.head()

,Source,Destination
1,0,1819
2,0,2840
3,1,1565
4,1,1309
5,1,1397


In [8]:
print(type(train['Source'][1]), type(train['Destination'][1]))

<class 'numpy.int64'> <class 'numpy.int64'>


In [9]:
nodes = sorted(list(set(train['Source'].values) | set (train['Destination'].values)))
print(len(nodes))
ledger = {}
for i, node in enumerate(nodes):
  ledger[node] = i
print(len(ledger))

4648
4648


In [10]:
friends = nx.DiGraph()
for src, dst in tqdm(train.values):
  friends.add_edge(ledger[src], ledger[dst])

  0%|          | 0/59382 [00:00<?, ?it/s]

In [11]:
df = pd.DataFrame({}, {}, columns = ['src', 'dest'])  
src_list = [ledger[src] for src,dest in train.values]
dest_list = [ledger[dest] for src, dest in train.values]
df['src'] = src_list
df['dest'] = dest_list
df['link'] = 1
print(df.head())

   src  dest  link
0    0  1819     1
1    0  2840     1
2    1  1565     1
3    1  1309     1
4    1  1397     1


In [12]:
df = df.sample(frac=1)

In [13]:
node1 = set(df['src'])
node2 = set(df['dest'])
df_nodes = node1.union(node2)
print(len(df_nodes))

4648


In [14]:
friends.number_of_nodes()

4648

In [15]:
friends.number_of_edges()

59382

In [16]:
for node in friends.nodes():
  if (len(friends.out_edges(node)) + len(friends.in_edges(node))) == 0:
    print(node)

in_out_zero = []
for node in friends.nodes():
  if len(friends.out_edges(node)) == 0 or len(friends.in_edges(node)) == 0:
    in_out_zero.append(node)

len(in_out_zero)

1219

In [17]:
nodelist = sorted(list(ledger.values()))
nodes = sorted(list(friends.nodes()))
print(len(nodelist))
if nodes == nodelist:
  print("true")
else:
  print("false")

4648
true


In [18]:
friends.adj[3]

AtlasView({3830: {}, 2504: {}, 3387: {}, 4528: {}, 246: {}, 1266: {}, 982: {}, 596: {}, 291: {}, 1819: {}, 2480: {}, 1676: {}, 1450: {}, 79: {}, 2112: {}, 222: {}, 2558: {}, 574: {}, 1376: {}, 2306: {}, 4642: {}, 1988: {}, 2922: {}, 896: {}})

# Section 1: Data Preparation

1.1 Extracting positive samples from the graph

The procedure followed is:
* Obtain the adjacency matrix
* Traverse to find nodes that are not connected
* Store these pairs of nodes to use as negative samples during training

In [19]:
list(friends.edges())[:5]

[(0, 1819), (0, 2840), (1819, 3107), (1819, 3431), (1819, 3432)]

In [20]:
def get_neg_samples(nodes, graph):
  """
  Input: 
    nodes: list of all nodes of the graph
    graph: the nx graph
  Output:
    neg_samples: a pd dataframe containing all pairs of disconnected nodes
  
  Computes the adjacency matrix, and returns all disconnected nodes
  """

  edges = set(graph.edges())

  discon_pairs = set([])

  while (len(discon_pairs)<59000):
    try:
      a=ledger[random.randint(0, 4648)]
      b=ledger[random.randint(0, 4648)]
    except:
      continue
    if a!=b and (a,b) not in edges and (b,a) not in edges:
        try:
            if nx.shortest_path_length(graph,source=a,target=b) > 2: 
                discon_pairs.add((a,b))
            else:
                continue  
        except:  
                discon_pairs.add((a,b))              
    else:
        continue

  # for i in tqdm(range(adj_g.shape[0]-1)):
  #   for j in range(i+1, adj_g.shape[1]):
  #     try:
  #       if nx.shortest_path_length(graph, i, j) <= 2:
  #         if adj_g[i, j] == 0:
  #           discon_pairs.append([nodes[i], nodes[j]])
  #     except nx.NetworkXNoPath:
  #       continue

  print(f"Number of disconnected nodes found = {len(discon_pairs)}")

  #converting the list into a dataframe

  neg_samples = pd.DataFrame({'src': [i[0] for i in discon_pairs], 
                              'dest': [i[1] for i in discon_pairs]})  
  neg_samples['link'] = 0

  return neg_samples

In [21]:
%%time
neg_data = get_neg_samples(nodelist, friends)

# estimated running time for this cell = 4 days
# check for possible places for optimization 

Number of disconnected nodes found = 59000
CPU times: user 23.3 s, sys: 77.4 ms, total: 23.4 s
Wall time: 25.2 s


In [22]:
neg_data.head()

,src,dest,link
0,980,2649,0
1,1704,2093,0
2,2400,655,0
3,3451,3922,0
4,989,2453,0


1.2 Obtain positive samples from the original graph

These positive samples, along with the negative samples obtained above would be used for training the classifiers. The positive samples are simply **pairs of nodes between which a link actually exists in the original graph.**

The procedure followed is: 
* Check if dropping a node pair results in splitting of the graph (increases the number of conn components)
* Check if dropping the pair results in a reduction in the number of nodes
* If both the above constraints are fulfilled, then drop the node pair (edge) - store this node pair in a list of positive samples

In [23]:
# creating a dictionary of number of times a node occurs in the dict
src = list(df['src'])
dest = list(df['dest'])

src_dict = collections.Counter(src)
dest_dict = collections.Counter(dest)

print(len(src_dict), len(dest_dict))

4028 4049


In [26]:
def get_pos_samples(df, g):
  """
  Input:
    df: the original edgelist as a pd dataframe 
    g: the original networkx graph
    assumes: 
      nodes are specified as Source and Destination
      original Graph is a Digraph
      node indexing is corrected
  Output:
    pos_samples: dataframe containing all removable edges (positive samples)
  """
  df_temp = df.copy()
  gr = g.copy(as_view=False)
  node_count = len(gr.nodes())
  print(f"original node count = {node_count}")
  
  removable_links_idx = []
  
  cc = nx.number_weakly_connected_components(gr)
  print(f"Original number of connected components = {cc}")

  for c, i in tqdm(enumerate(df.index.values)):
    src, dest, link = df.loc[i].values
    
    gr.remove_edge(src,dest)

    if(gr.degree(src) == 0):
      gr.remove_node(src)
    if(gr.degree(dest) == 0):
      gr.remove_node(dest)
  
    if len(gr.nodes()) == node_count:
      removable_links_idx.append(i)
      df_temp = df_temp.drop(index=i)
    else:
      gr.add_edge(src,dest)
    
    if c == 6000:
      print(f"removable links index = {len(removable_links_idx)}")
      break
  
  pos_samples = df.loc[removable_links_idx]
  pos_samples['link'] = 1

  nodes_new = set(df_temp['src']).union(set(df_temp['dest']))
  print(f"New number of nodes = {len(nodes_new)}")

  return pos_samples, df_temp

In [27]:
pos_samples, df_train = get_pos_samples(df, friends)
pos_samples_idx = pos_samples.index.values
print(pos_samples.shape)
pos_samples.head()

original node count = 4648
Original number of connected components = 1


0it [00:00, ?it/s]

removable links index = 5966
New number of nodes = 4648
(5966, 3)


,src,dest,link
50681,2767,2940,1
36993,1806,4398,1
45209,2357,3559,1
6383,277,4080,1
59289,4495,3544,1


In [28]:
print(df_train.shape)
df_train.head()

(53416, 3)


,src,dest,link
4836,201,4373,1
57767,3839,3890,1
8723,401,2124,1
33723,1585,3413,1
35066,1687,2607,1


In [29]:
# sanity check
train_nodes = set(df_train['src']).union(set(df_train['dest']))

In [30]:
len(df_nodes.difference(train_nodes))

0

In [31]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.MultiDiGraph())

In [32]:
original_graph = nx.from_pandas_edgelist(df, source="src", target="dest", create_using=nx.MultiDiGraph)

In [33]:
cc_before = nx.number_weakly_connected_components(original_graph)
print(cc_before)

1


In [34]:
cc_after = nx.number_weakly_connected_components(train_graph)

In [35]:
print(cc_after)

2


In [36]:
neg_data.shape

(59000, 3)

In [37]:
neg_test = neg_data.sample(frac = 0.1)
print(neg_test.shape)
neg_test.head()

(5900, 3)


,src,dest,link
36538,3384,1643,0
20199,3356,1415,0
43347,2441,1226,0
58610,1159,1991,0
22915,4319,143,0


In [38]:
neg_train = neg_data.drop(neg_test.index.values, axis=0)
print(neg_train.shape)
neg_train.head()

(53100, 3)


,src,dest,link
0,980,2649,0
1,1704,2093,0
2,2400,655,0
3,3451,3922,0
4,989,2453,0


In [39]:
combined_tr = pd.concat([df_train, neg_train], axis = 0).sample(frac=1)
print(combined_tr.shape)
combined_tr.head()

(106516, 3)


,src,dest,link
57144,3719,4620,1
41947,2091,913,1
20483,2434,3890,1
33112,2289,722,0
42911,2060,4570,0


In [40]:
combined_te = pd.concat([pos_samples, neg_test], axis = 0).sample(frac=1)
print(combined_te.shape)
combined_te.head()

(11866, 3)


,src,dest,link
33357,1572,4495,1
26115,3704,3624,0
7450,823,3889,1
26247,3579,2999,1
45224,2357,2922,1


In [41]:
combined_tr.to_csv(data_path+'/es_train_91.csv')
combined_te.to_csv(data_path+'/es_test_91.csv')

In [42]:
df_train.to_csv(data_path+'/es_pos_train_91.csv')

In [43]:
combined_tr = pd.read_csv(data_path+'/es_train_91.csv', index_col = [0])
print(combined_tr.shape)
combined_tr.head()

(106516, 3)


,src,dest,link
57144,3719,4620,1
41947,2091,913,1
20483,2434,3890,1
33112,2289,722,0
42911,2060,4570,0


In [44]:
combined_te = pd.read_csv(data_path+'/es_test_91.csv', index_col = [0])
print(combined_te.shape)
combined_te.head()

(11866, 3)


,src,dest,link
33357,1572,4495,1
26115,3704,3624,0
7450,823,3889,1
26247,3579,2999,1
45224,2357,2922,1


In [45]:
df_train = pd.read_csv(data_path+'/es_pos_train_91.csv', index_col = [0])
print(df_train.shape)
df_train.head()

(53416, 3)


,src,dest,link
4836,201,4373,1
57767,3839,3890,1
8723,401,2124,1
33723,1585,3413,1
35066,1687,2607,1


In [46]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.MultiDiGraph)

In [ ]:
scales = [5, 10, 20, 50]
sample_points = np.linspace(0, 100, 50).astype(np.float32)
degree = 20
G = StellarGraph.from_networkx(train_graph)
generator = GraphWaveGenerator(G, scales=scales, degree=degree)

In [ ]:
emb = generator.flow(
    node_ids = G.nodes(), sample_points = sample_points, batch_size = 1, repeat = False
)

In [ ]:
embeddings = [x.numpy() for x in tqdm(emb)]

  0%|          | 0/75879 [00:00<?, ?it/s]

In [ ]:
filename = data_path + "_embeddings"

In [ ]:
outfile = open(filename, "wb")
pickle.dump(embeddings, outfile)

In [ ]:
print(filename)

gdrive/My Drive/Major Project_embeddings


In [ ]:
file = open(filename, "rb")
gw_emb = pickle.load(file)
file.close()

EOFError: ignored

In [ ]:
print(len(gw_emb))

75879


In [ ]:
emb_dict = {}
for node, emb in zip(G.nodes(), gw_emb):
  emb_dict[node] = emb
print(len(emb_dict))

75879


In [ ]:
# get training embeddings
t_e = []
for i, row in combined.iterrows():
  comb_emb = emb_dict[row['src']] + emb_dict[row['dest']]
  t_e.append(comb_emb)
print(len(t_e))

999165


In [ ]:
t_y = combined['link']
len(t_y)

999165

In [ ]:

import xgboost as xgb
